In [ ]:
%%capture
!python -m spacy download en

In [ ]:
from datascience import *
import spacy
from collections import Counter
import folium
import numpy as np
from IPython.display import HTML, display, IFrame
from scripts.hist_module import * 
import requests
from datetime import datetime
import time
import urllib
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use("fivethirtyeight")

# Introduction to Importing Data, Using Tables and Creating Graphs 

## The Jupyter Notebook

First of all, note that this page is divided into what are called "cells". For example, the following cell is a "code cell" where you will write your code. You'll see a `In [ ]:` next to each cell for code, which is a counter for the cells you have run. You can navigate cells by clicking on them or by using the up and down arrows. Cells will be highlighted as you navigate them.

In [ ]:
# this is a code cell

### Executing cells

<p></p>

<div class="alert alert-info">
You can execute cells with <b><code>Ctrl-Enter</code></b> (which will run the cell and keep the same cell selected), or <b><code>Shift-Enter</code></b> (which will run the cell and then select the next cell).
</div>

Try running the following cell and see what it prints out:

In [ ]:
print("Hello world!")

## Creating Tables

### From Scratch

If we don't have a spreadsheet file and are starting with nothing, first we need to make arrays. Arrays are simply a form of list that a programming language uses to denote a collection of items. In the case of a table, we'll consider an array as either a row or a column. Let's make two arrays below that will become our columns, one for famous psychologists and one for the year they were born. 

In general, to make an array we use: 

```python
make_array(attribute_1, attribute_2, ...)
```

We set each of these created arrays equal to a variable name. This means that from now on, we can use that variable name to reference its respective array! Variables make information storage and retrieval much easier. 

In [ ]:
psychologist_names = make_array("Freud", "Skinner", "Piaget", "Maslow")
psychologist_birth = make_array(1856, 1904, 1896, 1908)

Since we've assigned these to variables, all we have to do is call the variable name to get the information back, or to manipulate it!

In [ ]:
psychologist_names

Now, to make a table using these arrays, we use the general form:

```python
Table( ).with_columns("Column Name", array_name, . . .)
```

We assign the created table to a variable (just like the arrays from above), and then type that variable name to display the table. 

In [ ]:
psych_table= Table().with_columns("Psychologist", psychologist_names,
                                  "Birth Year", psychologist_birth)
psych_table.show()

### Importing

It's more likey that a file holding your data already exists. In general, to import data from a file, we write:

```python
Table.read_table("file_name")
```

Most often, these file names end in `.csv` to show the data format. `.csv` format is popular for spreadsheets and can be imported/exported from programs such as Microsoft Excel, OpenOffice Calc, or Google spreadsheets.

We've scraped some data on [job postings](http://careers.historians.org/jobs/?page=1) for historians from the American Historical Association:

In [ ]:
job_data = Table.read_table('data/AHA-jobs.csv')

Let's `show` the first 5 rows:

In [ ]:
job_data.show(50)

Python can calculate how large this table is with two functions: `num_rows` and `num_columns`. The general form for these functions are `table.num_rows` and `table.num_columns`. 

Let's use these on the table above. 

In [ ]:
job_data.num_rows

In [ ]:
job_data.num_columns

It looks like we have 144 job postings, and 14 different columns of metadata for each posting.

There are two methods to subset a table with select columns. We could either use the 'select' function or the 'drop' function. 

- `select` can create a new table with only the columns indicated in the parameters 
- `drop` can create a new table with columns NOT indicated in the parameters

In [ ]:
job_data.select('title')

In [ ]:
job_data.select('title', 'employer')

If we want to select only a specific subset of rows, we can use the `where` method. The general form of this function is:

```python
table_name.where(column_name, predicate)
```

Let's look only at job postings `where` the `primary_field` is `equal_to` "United States/North America":

In [ ]:
job_data.where("primary_field", are.equal_to('United States/North America'))

---

### Tables Essentials!

For your reference, here's a table of useful `Table` functions:

|Name|Example|Purpose|
|-|-|-|
|`Table`|`Table()`|Create an empty table, usually to extend with data|
|`Table.read_table`|`Table.read_table("my_data.csv")`|Create a table from a data file|
|`with_columns`|`tbl = Table().with_columns("N", np.arange(5), "2*N", np.arange(0, 10, 2))`|Create a copy of a table with more columns|
|`column`|`tbl.column("N")`|Create an array containing the elements of a column|
|`sort`|`tbl.sort("N")`|Create a copy of a table sorted by the values in a column|
|`where`|`tbl.where("N", are.above(2))`|Create a copy of a table with only the rows that match some *predicate*|
|`num_rows`|`tbl.num_rows`|Compute the number of rows in a table|
|`num_columns`|`tbl.num_columns`|Compute the number of columns in a table|
|`select`|`tbl.select("N")`|Create a copy of a table with only some of the columns|
|`drop`|`tbl.drop("2*N")`|Create a copy of a table without some of the columns|
|`take`|`tbl.take(np.arange(0, 6, 2))`|Create a copy of the table with only the rows whose indices are in the given array|
|`join`|`tbl1.join("shared_column_name", tbl2)`|Join together two tables with a common column name
|`are.equal_to()`|`tbl.where("SEX", are.equal_to(0))`|find values equal to that indicated|
|`are.not_equal_to()`|`tbl.where("SEX", are.not_equal_to(0))` | find values not including the one indicated|
|`are.above()`| `tbl.where("AGE", are.above(30))` | find values greater to that indicated|
|`are.below()`| `tbl.where("AGE", are.below(40))` | find values less than that indicated |
|`are.between()`| `tbl.where("SEX", are.between(18, 60))` | find values between the two indicated |

---

## Visualizations 

Now that we have a manageable table we can start making visualizations! We're going to make our data easier to work with by using the `group()` function. This function will group all the data from a specific column by the amount of times it shows up throughout the table 

An example is shown below:

In [ ]:
job_data.group('primary_field')

Let's organize the count table so that it puts the entries with the highest counts at the top of the table. We will do this by adding the `descending=True` parameter. 

In [ ]:
job_data.group('primary_field').sort('count', descending=True)

It's time to make our first visualization. We are going to create a bar graph of the data in the above cell, since it is grouped by counts. 

To create a bar graph, we need to use either the `bar()` or `barh()` function. The added `h` in `barh()` makes the graph appear horizontally instead of vertically. The general form is:

```python
table.barh("column to graph")
```

In [ ]:
job_data.group('primary_field').barh('primary_field')

Great! Let's create some more bar graphs for other columns in the data. We can get the names with the `column_labels` method.

In [ ]:
job_data.column_labels

## Challenge

Use the `group` and `barh` methods to visualize some of the other categorical variables.

In [ ]:
# YOUR CODE HERE

---

## Basic Data Analysis 

Let's use some of the results from above to perform an exploratory data analysis of our job listing data. 

It is important to note that performing quantitative data analysis on data sets such as the job listings set that we are using is quite difficult. This is due to the fact that the data doesn't have any continuous numerical variables that we can really use to see trends and correlations. In other words, most of our data is general qualitative descriptions such as job descriptions or specific job titles. The `salary` column is a potential place of quantitative analysis interest, but unfortunately most jobs list the salary as 'Open', making the analysis less meaningful.

However, we can still try to analyze our data quantitatively. We just need to play around with it. 

Specifically, let's see if there is a correlation between job type (Experienced, Entry Level, Internship) and preferred education level (Doctorate, Master, None). To do this, we will assign numerical values to represent each category. Higher numerical values will correspond to higher positions/educations.

The cell below goes through each row in the table, and stores its `preferred_education` and `type` information in two seperate lists. Think of the `education_points` list as a list of x-coordinates and the `type_points` list as a list of y-coordinates. 

Our goal is to use those lists to eventually plot the data and look for a correlation

In [ ]:
def get_weekday(x):
    return datetime.strptime(x, '%B %d, %Y').weekday()

In [ ]:
get_weekday('November 14, 2017')

In [ ]:
job_data.append_column(label="day_posted", values=[get_weekday(x) for x in job_data['date_posted']])
job_data.show(5)

In [ ]:
job_data.group('day_posted').sort('count', descending=True)

In [ ]:
gs = job_data.groups(['day_posted', 'primary_field'])

In [ ]:
gs

In [ ]:
gs.where('primary_field', are.equal_to('Africa'))

In [ ]:
gs.where('primary_field', are.equal_to('Europe'))

In [ ]:
gs.where('primary_field', are.equal_to('Asia'))

In [ ]:
gs.where('primary_field', are.equal_to('Middle East/North Africa'))

In [ ]:
job_data.group('day_posted').barh('day_posted')

We are going to find the 'correlation coefficient' of this data. 

---

### The correlation coefficient - *r*

> The correlation coefficient ranges from −1 to 1. A value of 1 implies that a linear equation describes the relationship between X and Y perfectly, with all data points lying on a line for which Y increases as X increases. A value of −1 implies that all data points lie on a line for which Y decreases as X increases. A value of 0 implies that there is no linear correlation between the variables. ~Wikipedia

---

*r* = 1: the scatter diagram is a perfect straight line sloping upwards

*r* = -1: the scatter diagram is a perfect straight line sloping downwards.

Let's calculate the correlation coefficient between preferred education and job type. We can use the `np.corrcoef` function on the two variable (columns here) that we want to correlate:

In [ ]:
np.corrcoef(education_type['Education'], education_type['Type'])

This function returns a matrix for each variable. We have two 1s because, of course, each variable is perfectly correlated with itself. Our coefficient here is around -0.17, which indicates almost no correlation. That tells us that at least for this data set, jobs with higher positions don't neccessarily necessitate the highest degrees of education. 

We can also create a quick scatter plot of our table data. Note that the scatter plot for this data isn't actually too useful, since many values in the table are repeated. 

To create a bar graph, we need to use either the `scatter()` function. The general form is:

```python
table.scatter("column for x-axis", "column for y-axis")
```

In [ ]:
education_type.scatter('Education', 'Type')

As stated above, the scatter plot doesn't really provide us with too much information. When performing data analysis, it is always important to try a lot of different analysis/visualization techniques, since each technique has its pros and cons and excels in different situation. In this case, our correlation coefficient analysis gave us more information than the scatter plot.

---

## Text Analysis

We can also perform data analysis with the textual information in our dataset. Based on the non-numerical style of our dataset, we may be able to make more meaningful conclusions about our data. 

We are going to be using the `spaCy` library for our text analysis. The package contains a lot of powerful functions for large-scale text analysis and natural language processing.

The first thing we are going to do is load in English as our language.

In [ ]:
nlp = spacy.load('en')

Let's take a look at some of the textual information in our table. Below, we look at the first entry in the `job_description` column in our table.

In [ ]:
job_data['job_description'][0]

We are going to use spaCy's `nlp` function to analyze the paragraph from the cell above. The `nlp` function is very powerful, and has the ability to break up a text into sentences/phrases. `parsed_text` will be the variable that contains our information.

In [ ]:
parsed_text = nlp(str(job_data['job_description'][0]))

Below, we print out the length of the parsed text as well as the number of sentences that it contains

In [ ]:
len(parsed_text)

In [ ]:
len(list(parsed_text.sents))

We know that the `parsed_text` variable contains information about each of the sentences in our original paragraph. We are going to take that information and turn it into a table to perform further analysis. The code in the cell below will go through every sentence that the `parsed_text` variable contains and put each one in a new row in our table

In [ ]:
sents_tab = Table()
sents_tab.append_column(label="Sentence", values=[sentence.text for sentence in parsed_text.sents])
sents_tab.show()

We are going to do the same thing once again, however with words this time instead of sentences. The `parsed_text` variable also contains information about the individual words in our original paragraph.

In [ ]:
toks_tab = Table()
toks_tab.append_column(label="Word", values=[word.text for word in parsed_text])
toks_tab.show()

Let's add another column to our table of words. The `parsed_text` variable from the `nlp` function earlier also contains information about each word's part of speech. We are going to append that information to our table.

In [ ]:
toks_tab.append_column(label="POS", values=[word.pos_ for word in parsed_text])
toks_tab.show()

We are going to add one more column to the table from the above cell. This column is going to contain the 'Lemma' of each word. The "Lemma" of a word is that word's 'base form'. Once again, we will query the `parsed_text` variable for this information

In [ ]:
toks_tab.append_column(label="Lemma", values=[word.lemma_ for word in parsed_text])
toks_tab.show()

It's great that we have all this data in individual tables. However, it would be most useful to have it all in one table. We define a function below that takes in a variable like `parsed_text` and outputs one table that contains all the information from the last few cells. 

In [ ]:
def tablefy(parsed_text):
    toks_tab = Table()
    toks_tab.append_column(label="Word", values=[word.text for word in parsed_text])
    toks_tab.append_column(label="POS", values=[word.pos_ for word in parsed_text])
    toks_tab.append_column(label="Lemma", values=[word.lemma_ for word in parsed_text])
    toks_tab.append_column(label="Stop Word", values=[word.is_stop for word in parsed_text])
    toks_tab.append_column(label="Punctuation", values=[word.is_punct for word in parsed_text])
    toks_tab.append_column(label="Space", values=[word.is_space for word in parsed_text])
    toks_tab.append_column(label="Number", values=[word.like_num for word in parsed_text])
    toks_tab.append_column(label="OOV", values=[word.is_oov for word in parsed_text])
    toks_tab.append_column(label="Dependency", values=[word.dep_ for word in parsed_text])
    return toks_tab

Let's run our `tablefy()` function on our original `parsed_text` variable.

In [ ]:
tablefy(parsed_text).show()

That's really cool! All of our extremely detailed information about that one entry is now in one place. We can now use the `where` method we saw above to explore the data more. Here's an example:

In [ ]:
adjectives = tablefy(parsed_text).where('POS', are.equal_to('ADJ'))
adjectives.show(10)

## Challenge

Use the syntax above to explore this first job description more:

In [ ]:
# YOUR CODE HERE

---

# Aggregating Text

Now, let's get the same type of information for all the entries in the `job_description` column. Recall that the table above only contains the information about the first entry. 

In order to get information from the whole column, we are going to pass it in entirely to the `nlp` function. If you look at the output, you'll see that over 60,000 rows are omitted (from being displayed). That's a lot of data!

In [ ]:
all_descriptions = tablefy(nlp(' '.join(job_data['job_description'])))
all_descriptions.show(100)

We can now perform more analysis using some of the useful `Table()` functions from the previous sections. Let's first subset our table by only selecting the rows where the `POS` column is equal to `ADJ`. In other words, we will only keep the words that are adjectives.

In [ ]:
adjectives = all_descriptions.where('POS', are.equal_to('ADJ'))
adjectives.show(10)

Now that we have our smaller table, let us take counts of how many times each word appears, and display that information.

In [ ]:
Counter(adjectives['Word']).most_common()

Notice that some of the most common words in our display above don't really provide us with too much information. For example, the words 'their' and 'that' appear very often in our data, but don't really provide us with insight into the data since they are simply just common words that are used in the English language. It would be ideal if we removed them from our data in order to only keep the words that are actually useful and significant.

Fortunately, `spaCy` has functionality to do this. In the `Stop Word` column, each word either has a True or a False depending on whether it is one of the most common words in the language (part of a stop list). We are going to subset our data again by picking out the adjectives. However this time, we are only going to keep the adjectives that are not stop words (i.e that are not very common English words like 'their' and 'that').

In [ ]:
adjectives = all_descriptions.where('POS', are.equal_to('ADJ')).where('Stop Word', are.equal_to(False))
Counter(adjectives['Word']).most_common()

We can also search for words within the 'count' information. If I am interested in the amount of times that the word 'digital' shows up in the table, I can query the Counter for that word. 

In [ ]:
Counter(adjectives['Word'])['digital']

## Challenge

Tr exploring what nouns and verbs are used most frequently in these job descriptions. Once again, you may want to leave out the `Stop Word`s:

In [ ]:
# YOUR CODE HERE

## N-grams

N-grams help us move beyond looking at one word, and allow us to look at co-occurence. It takes window snapshots of a list of words going one word at a time. It's easiest to understand from an example. First we'll define a function `find_ngrams`, don't worry about the code here!

In [ ]:
def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

We'll look at a bigram, with a window of 2:

In [ ]:
list(find_ngrams('I like walking my dog'.split(), 2))

Here's a trigram:

In [ ]:
list(find_ngrams('I like walking my dog'.split(), 3))

Let's take our aggregated job description data, remove `Stop Word`s and `Punctuation`, and try out some ngram counts:

In [ ]:
sifted_words = all_descriptions.where('Stop Word', are.equal_to(False)).where('Punctuation', are.equal_to(False))['Word']
sifted_words

In [ ]:
Counter(find_ngrams(sifted_words, 3)).most_common()

## Challenge

Try working with ngrams on the `employer_about` column!

In [ ]:
# YOUR CODE HERE

## Mapping

Mapping requires a bit more coding knowledge, but we'll just show a quick example here of how little you have to write to generate a cool map!

```python
latitude = []
longitude = []
for i in range(job_data.num_rows):
    search = urllib.parse.quote(job_data['employer'][i])
    
    print(job_data['employer'][i])

    try:
        json_res = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(search)).json()
        coordinates = json_res['results'][0]['geometry']['location']
        latitude.append(coordinates['lat'])
        longitude.append(coordinates['lng'])
    except:
        latitude.append('')
        longitude.append('')

    time.sleep(.5)
```

In [ ]:
job_data = job_data.with_columns('latitude', latitude, 'longitude', longitude)
job_data.show(5)

First we'll sort out the `employment_type` by color:

In [ ]:
color_dict, html_key = assign_colors(job_data.to_df(), "employment_type")
display(HTML(html_key))

Then we can map each of the jobs by their employment type:

In [ ]:
mapa = folium.Map(location=[39.8333333,-98.585522], zoom_start=3) # Folium is a useful library for generating
                                                                   # Google maps-like map visualizations.
for r in job_data.rows:
    
    if r[-2] != '':
        folium.CircleMarker((float(r[-2]), float(r[-1])),
                    radius=1,
                    popup=r[10],
                    color=color_dict[r[10]],
                    fill_color=color_dict[r[10]],
                   ).add_to(mapa)

mapa.save("map1.html")
IFrame('map1.html', width=700, height=400)